# LlamaIndex Router Engine

In [41]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

In [38]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(
    input_files=["metagpt.pdf"]
).load_data()

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [18]:
# Define LLM and Embedding model
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm = Ollama(model="llama3", request_timeout=120)
Settings.embed_model = OllamaEmbedding(model_name="llama3")
# resp = llm.complete("who is the president of the united states")

In [19]:
# Define Summary Index and Vector Index over the Same Data
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [22]:
# Define Query Engines and Set Metadata
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [46]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization questions related to state_of_the_union"),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieving specific context from the state_of_the_union."),
)

In [47]:
# Define Router Query Engine
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True,
)

In [48]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Since the question is asking what the summary of the document is, option 1 'Useful for summarization questions related to state_of_the_union' seems most relevant..
The document appears to be a speech by the President, discussing the state of the economy and the need for Congress to pass certain bills to make record investments in emerging technologies and American manufacturing. The President highlights the importance of making more cars and semiconductors in America, cutting energy costs, and reducing the cost of child care. They also propose closing loopholes so corporations and the wealthiest Americans start paying their fair share of taxes.


In [49]:
response = query_engine.query("Who is the president of the United States?")
print(str(response))

Selecting query engine 1: No specific information about the president of the United States can be found in a state_of_the_union summary.
The speaker being referred to in the given text appears to be a President of the United States.


In [54]:
from IPython.display import Markdown, display


# define prompt viewing function
def display_prompt_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}" f"**Text:** "
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown(""))


prompts_dict = query_engine.get_prompts()

display_prompt_dict(prompts_dict)

**Prompt Key**: summarizer:summary_template**Text:** 

Context information from multiple sources is below.
---------------------
{context_str}
---------------------
Given the information from multiple sources and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt Key**: selector:prompt**Text:** 

Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.
---------------------
{context_list}
---------------------
Using only the choices above and not prior knowledge, return the choice that is most relevant to the question: '{query_str}'

